# Hipotezy i wnioski

### Sprawdzanie hipotez

W klasycznym podejściu mamy hipotezę zerową H0, która stanowi domyślny punkt wyjścia, a także alternatywną hipotezę H1, i chcemy je porównać. Dzięki parametrom statystycznym możemy określić cz hipotezę H0 można odrzucić jako fałszywą. 

### Przykład: rzut monetą

Chcemy sprawdzić czy posiadana przez nas moneta jest "uczciwa". 

Jeśli założymy, żę p to prawdopodobieństwo wyrzucenia tą monetą orła, to naszą hipotezą zerową jest to, że moneta jest uczciwa - tj. p=0,5. Hipotezę tę porównamy z alternatywną hipotezą - p ≠ 0,5.

W praktyce test będzie polegał na wykonaniu n rzutów monetą i policzeniu liczby wyrzuconych orłów (X). Każdy rzut monetą jest próbą Bernoulliego, a więc X jest zmienną losową o rozkładzie dwumianowym (n,p)

In [1]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Określa wartości mi i sigma."""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

Jeśli zmienna losowa charakteryzuje się rozkładem normalnym, to możemy skorzystać z funkcji normal_cdf w celu określenia prawdopodobieństwa tego, że wartość znajduje się w obrębie jakiegoś interwału lub poza nim:

In [3]:
def normal_cdf(x: float, mi: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mi) / math.sqrt(2) / sigma)) / 2

# Funkcja dystrybuanty rozkładu normalnego określa prawdopodobieństwo tego, że zmienna znajduje się poniżej wartości progowej.
normal_probability_below = normal_cdf

# Jeżeli nie znajduje się nad wartością progową, to znajduje się pod nią.
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """Prawdopodobieństwo tego, że N(mi, sigma) jest większe niż lo."""
    return 1 - normal_cdf(lo, mu, sigma)

# Wartość znajduje się w przedziale, jeżeli jest mniejsza od górnej wartości granicznej i większa od dolnej wartości granicznej.
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """Prawdopodobieństwo tego, że N(mi, sigma) jest pomiędzy lo i hi."""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# Wartość jest poza przedziałem, jeżeli nie znajduje się pomiędzy ograniczeniami.
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """Prawdopodobieństwo tego, że N(mi, sigma) nie jest pomiędzy lo i hi."""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

Możemy teraz wykonać odwrotną operację - znaleźć niekońcowy obszar lub (symetryczny) interwał wokół średniej, który odpowiada pewnemu poziomowi prawdopodobieństwa. Jeśli chcemy określić interwał wyśrodkowany na wartości średniej i zawierajacy 60% prawdopodobieństwa, to musimy określić wartości odcięcia, w których dolne i górne krańcowe obszary zawierają po 20% prawdopodobieństwa (pozostawiajac środkowe 60%).

In [4]:
def inverse_normal_cdf(p: float,
                       mi: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:
    """Znajduje przybliżoną wartość odwrotności przy użyciu algorytmu wyszukiwania binarnego."""

    # Jeżeli rozkład nie jest standardowy, to oblicz jego standardową postać i przeskaluj.
    if mi != 0 or sigma != 1:
        return mi + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      # normal_cdf(-10) ma wartość (zbliżoną do) 0
    hi_z  =  10.0                      # normal_cdf(10)  ma wartość (zbliżoną do) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # Weź pod uwagę punkt środkowy
        mid_p = normal_cdf(mid_z)      # i znajdującą się tam wartość dystrybuanty.
        if mid_p < p:
            low_z = mid_z              # Punkt środkowy znajduje się za nisko, szukaj nad nim.
        else:
            hi_z = mid_z               # Punkt środkowy znajduje się za wysoko, szukaj pod nim.

    return mid_z

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Zwraca z przy zachowaniu warunku P(Z <= z) = prawdopodobieństwo."""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Zwraca z przy zachowaniu warunku P(Z >= z) = prawdopodobieństwo"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    """
    Zwraca granice symetryczne (umieszczone wokół średniej), 
    które obejmują określone prawdopodobieństwo.
    """
    tail_probability = (1 - probability) / 2

    # Nad górną granicą powinna znajdować się wartość tail_probability.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # Pod dolną granicą powinna znajdować się wartość tail_probability.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

Aby nasza hipoteza o uczciwości monety była prawdziwa w przypadku, gdy chcemy rzucić monetą n=1000, rozkład zmiennej X powiniem być zbiżony do rozkładu normalnego o średniej równej 50 i odchyleniu standardowym równym 15,8

In [5]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [6]:
sigma_0

15.811388300841896

Musimy podjąć decyzję dotyczącą istotności - jak bardzo skłonni jesteśmy popełniać błąd typu 1 ("fałszywie pozytywny"), w którym odrzucamy hipotezę H0 nawet wtedy, gdy jest prawdziwa. Z powodów historycznych przyjmuje się, że chęć popełnienia tego błędu jest określana na 5% lub 1%. Przyjmijmy wartość 5%.

Przyjrzyjmy się testowi odrzucającemu hipotezę H0, jeśli X znajduje się poza granicami wyznaczonymi przez:

In [7]:
# (469, 531)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [8]:
lower_bound

469.01026640487555

In [9]:
upper_bound 

530.9897335951244

In [10]:
assert 468.5 < lower_bound < 469.5
assert 530.5 < upper_bound < 531.5

Przy założeniu, że p jest naprawdę równe 0,5 (hipoteza H0 jest prawdziwa), istnieje tylko 5% szans na zaobserwowanie wartości X znajdującej się poza tym interwałem.

Patrząc na to z innej stronym jeśli hipoteza H0 jest prawdziwa, to wtedy około 19 na 20 razy test da poprawny wynik.

Moc testu - prawdopodobieństwo niepopełnienia błędu typu 2, w którym nie odrzuca się hipotezy H0 nawet wtedy, gdy jest ona fałszywa. 

W celu zmierzenia tego parametru musimy określić, co dokładnie oznacza fałszywość hipotezy H0.

Sprawdźmy, co się dzieje, jeżeli p wynosi 0,55 (podczas rzucania monetą częściej wypada orzeł)

In [11]:
# Ograniczenie 95% na podstawie założenia, że p jest równe 0,5.
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# Rzeczywiste wartości mi i sigma przy założeniu, że p = 0,55.
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# Błąd typu drugiego oznacza błąd polegający na nieodrzuceniu hipotezy zerowej,
# do czego dochodzi, gdy X wciąż znajduje się w naszym początkowym interwale.
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability      # 0.887

In [12]:
power

0.8865480012953671

In [13]:
assert 0.886 < power < 0.888

Załóżmy, że naszą hipotezą zerową jest to, że moneta nie jest zrobiona tak, że częściej wypada na niej orzeł (p<=0,5). W takim przypadku chcemy sprawdzić test jednostronny, który odrzuca hipotezę H0, gdy wartość X jest o wiele większa od 500, ale nie wtedy, gdy jest mniejsza od 500.
W związku z tym test 5% istotności wiąże się z koniecznością zastosowania funkcji narmal_probability_below w celu określenia punktu, poniżej którego znajduje się 95% prawdopodobieństwa

In [14]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
# Wynosi 526 (< 531, ponieważ potrzebujemy więcej prawdopodobieństwa w górnej części).

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability      # 0.936

In [15]:
hi

526.0073585242053

In [16]:
power

0.9363794803307173

In [17]:
assert 526 < hi < 526.1
assert 0.9363 < power < 0.9364

To test o większej wadze, ponieważ nie odrzucamy już hipotezy H0, gdy wartość X jest mniejsza od 469 (co jest bardzo mało prawdopodobne, jeśli hio=poteza H1 jest prawdziwa), ale odrzucamy hipotezę H0, gdy wartość X znajduje się w przedziale od 526 do 531 (co jest raczej prawdopodobne, jeżeli hipoteza H1 jest prawdziwa).

### Wartość p

Prawdopodobieństwo przy założeniu, że hipoteza H0 jest prawdziwa

In [18]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    Jak prawdopodobne jest zaobserwowanie wartości przynajmniej tak skrajnej jak x
    (w obydwu kierunkach) jeżeli wartości są z przedziału N(mi, sigma)?
    """
    if x >= mu:
        # Jeżeli x jest większy od średniej…
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # Jeżeli x jest mniejszy od średniej….
        return 2 * normal_probability_below(x, mu, sigma)

# Gdybyśmy wyrzucili orła 530 razy
two_sided_p_value(529.5, mu_0, sigma_0)   # 0.062

0.06207721579598835

In [22]:
# symulacja
import random

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # Policz liczbę wyrzuconych orłów
                    for _ in range(1000))                # podczas 1000 rzutów monetą.
    if num_heads >= 530 or num_heads <= 470:             # Oblicz, ile razy liczba ta
        extreme_value_count += 1                         # osiąga „ekstremum”.

extreme_value_count 

64

In [23]:
# wartość p wynosi 0,062, więc około 62 skrajne wartości z 1000
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

Wartość p jest wyższa od 5% istotności, a więc nie odrzucamy hipotezy zerowaj, Jeżeli zaobserwowalibyśmy wyrzucenie orła 532 razy, to wartość p wynosiłaby:

In [26]:
two_sided_p_value(531.5, mu_0, sigma_0)   # 0.0463

0.046345287837786575

a więc byłaby niższa od 5% istotności i musielibyśmy odrzucić hipotezę zerową. To dokładnie taki sam test jak ten co przeprowadzaliśmy wcześniej, Jedyną różnicą jest podejście do parametrów statycznych.

W obu przypadkach zachodzą równości:

In [27]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

Gdybyśmy zaobserwowami wyrzucanie orła 525 razy:

In [28]:
upper_p_value(524.5, mu_0, sigma_0) # 0.061

0.06062885772582072

In [29]:
# orzeł 527 razy
upper_p_value(526.5, mu_0, sigma_0) # 0.047, odrzucilibyśmy hipotezę zerową

0.04686839508859242

### Przedziały ufności

Możemy np. oszacować prawdopodobieństwo tego, że moneta jest nieuczciwa, przyglądając się średniej wartości zmiennych Bernoulliego odpowiadajacyh poszczególnym rzutom (1 oznacza wyrzucenie orła, a 0 reszki). Jeżeli wykonując 1000 rzutów, zaobserwujemy wyrzucenie orła 525 razy, to szacujemy, że p jest równe 0,525.

Jak bardzo możemy ufać temu szacunkowi? Gdybyśmy znali dokładną wartość parametru p, to korzystając z centralnego twierdzenia granicznego, moglibyśmy obliczyć średnie zmiennych Bernoulliego, które powinny charakteryzować się rozkładem normalnym, przy średniej p i odchyleniu standardowym: math.sqrt(p * (1 - p) / 1000)

W ym przypadku nie znamy wartości p, a więc możemy skorzystać z naszych szacunków:

In [30]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)   # 0.0158

In [31]:
sigma

0.015791611697353755

Korzystając z przybliżenia o rozkładzie normalnym, dochodzimy do wniosku, że "jesteśmy pewni na 95%" co do tego, że następujacy przedział zawiera prawdziwą wartość parametru p:

In [32]:
normal_two_sided_bounds(0.95, mu, sigma)        # [0.4940, 0.5560]

(0.4940490278129096, 0.5559509721870904)

Gdyby eksperyment został powtórzony wielokrotnie, to w 95% przypadków "prawdziwy" parametr (jest on za każdym razem taki sam) znajdowałby się w zaobserwowanym przedziale ufności (za każdym razem może on być inny).

W tym przypadku nie dochodzimy do wniosku, że moneta jest nieuczciwa, ponieważ wartość 0,5 mieści się  w przedziale ufności.

Gdybyśmy zaobserwowali wyrzucenie orła 540 razy, to:

In [33]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000) # 0.0158

In [34]:
sigma

0.015760710643876435

In [35]:
normal_two_sided_bounds(0.95, mu, sigma) # [0.5091, 0.5709]

(0.5091095927295919, 0.5708904072704082)

W takim przypadku "uczciwa moneta" nie mieści się w przedziale pewności. (Hipoteza "uczciwej monety" nie przechodzi testu- założyliśmy, że gdyby hipoteza była prawdziwa, to okazywałaby się prawdziwa w 95% przypadków).

### Hakowanie wartości p

Procedura, która błędnie odrzuca hipotezę zerową w tylko 5% przypadków, będzie zgodnie z definicją błędnie odrzucać hopotezę zerową przez 5% czasu:

In [36]:
from typing import List

def run_experiment() -> List[bool]:
    """Wykonaj 1000 rzutów monetą. True oznacza orła, a False reszkę."""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Zakłada 5% poziom ufności."""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

In [37]:
num_rejections

46

Wniosek z tego taki, że jeśli szukamy "istotnych" wyników, to zwykle można je znaleźć. Wystarczy przetestować wystarczająco dużo hipotez, a wśród nich z pewnością znajdą się takie, które wydają się być istotne. Po usunięciu odpowiednich obserwacji odstających wartość parametru p można zmniejszyć poniżej 0,05. Zabieg ten określa sie mianem hakowania wartości p. 

### Przykład: przeprowadzanie testu A-B

Przeprowadzono eksperyment polegający na losowym pokazywaniu osobom odwiedzającym stronę jednej z dwóch reklam i śledzeniu liczby kliknięć poszczególnych wersji reklamy.

Jeżeli 990 spośród 1000 użytkowników, którzy widzieli reklamę A, kliknęło ją, a tylko 10 z 1000 użytkowników, którzy widzieli reklamę B, kliknęło reklamę, to z pewnością można uznać, że reklama A jest lepsza.

Załóżmy, że N_a ludzi widziało reklamę A i n_a z nich ją kliknęło. Każde wyświetlenie reklamy można potraktować jako próbę Bernoulliego, w każdej p_a jest prawdopodobieństwem tego, że ktoś kliknie w reklamę A. Wtedy wiemy, że n_a/N_a jest zmienną losową o rozkładzie zbliżonym do normalnego, a więc o średniej p_a i odchyleniu standardowym sigma_a. Podobnie z B.

In [38]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

Jeżeli założymy, że te dwie zmienne o rozkładzie normalnym są niezależne, to ich różnica również powinna mieć rozkład normalny.

W związku z tym możemy sprawdzić hipotezę zerową, mówiącą, że wartości p_a i p_b są takie same.

In [39]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

Jeżeli założymy, że jedna reklama uzyska 200 kliknięć na 1000 odsłon, a druga uzyska 180 kliknięć na 1000 odsłon, to będziemy mieli do czynienia z następującą wartością parametru statystycznego:

In [40]:
z = a_b_test_statistic(1000, 200, 1000, 180)    # -1.14

In [41]:
z

-1.1403464899034472

Prawdopodobieństwo uzyskania tak dużej różnicy, gdyby średnie były w rzeczywistości równe wynosi:

In [42]:
two_sided_p_value(z)                            # 0.254

0.254141976542236

Wartość ta jest na tyle duża, że nie można orzec, iż różnica jest duża. Z drugiej strony, gdyby druga reklama uzyskała tylko 150 kliknięć, to uzyskalibyśmy następujące parametry:

In [43]:
z = a_b_test_statistic(1000, 200, 1000, 150)    # -2.94
z

-2.948839123097944

In [44]:
two_sided_p_value(z)                            # 0.003

0.003189699706216853

Oznacza to, że prawdopodobieństwo uzyskania tak dużej różnicy przy równie skutecznych reklamach wynosi zaledwie 0,003.

### Wnioski bayesowskie

Zaprezentowaneprocedury doprowadziły do utworzenia następującego stwierdzenia dotyczącego naszych testów: "istnieje tylko 3% szans na to, że zaobserwowanie takich parametrów statystycznych byłoby możliwe, gdyby hipoteza zerowa była prawdziwa".

Alternatywną techniką wnioskowania jest traktowanie nieznanych parametrów tak, jakby same były zmiennymi losowymi. Analityk rozpoczyna pracę od rozkładu a priori parametrów, a następnie korzysta z zebranych w ten sposób danych i twierdzenia Bayesa w cellu uzyskania zaktualizowanego rozkładu a posteriori paramentrów. Zamiast określać prawdopodobieństwo związane z testem, określane jest prawdopodobieństwo samych paramentrów. 

Jeżeli nieznany parametr jest prawdopodobieństwem korzystamy z a prori rozkładu beta, w którym wszystkie prawdopodobieństwa przyjmują wartości od 0 do 1:

In [45]:
def B(alpha: float, beta: float) -> float:
    """Stała normalizacji zapewniająca całkowite prawdopodobieństwo równe 1."""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # Wszystkie wagi mieszczą się w zakresie [0, 1].
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Jeśli parametry alpha i beta przyjmują wartość 1, to mamy do czynienia z rozkładem jednorodnym wyśrodkowanym w punkcie 0,5, który jest dość rozłożysty. Jeśli parametr alpha jest znacznie większy od beta, to większość wag przyjmuje wartości zbliżone do 1, a jeżeli parametr alpha jest znacznie mniejszy od beta, to większość wag przyjmuje wartości zbliżone do 0. 